<a href="https://colab.research.google.com/github/lucasleao03/GSI073---Topicos-Especiais-de-Inteligencia-Artificial-LLMs-Large-Language-Models-/blob/main/GSI073_aula0_luong_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação dos dados

Esta tarefa é inverter sequências de caracteres. Exemplo: **aabcd** em **dcbaa**.


In [ ]:
import torch
import torch.nn as nn
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt

chars = list("abcd ")
vocab = {ch: i for i, ch in enumerate(chars)} # Cada letra, ganha um número
inv_vocab = {i: ch for ch, i in vocab.items()}# Tabela de decodificação
vocab_size = len(vocab)

def encode(s): # Codifica letras em números
    return torch.tensor([vocab[c] for c in s], dtype=torch.long)

def decode(t): # Decodifica números em letras
    return ''.join(inv_vocab[int(x)] for x in t)

def random_seq(n=5): # Cria novas sequências
    return ''.join(random.choice(chars[:-1]) for _ in range(n))

# Gerar dados
pairs = [(encode(s), encode(s[::-1])) for s in [random_seq() for _ in range(50000)]]

max_len = max(len(x) for x, _ in pairs) # pega maior sequência

def pad(x):  # Preenche conjunto de dados em pad no último índice
    return torch.cat([x, torch.tensor([vocab[' ']] * (max_len - len(x)))], dim=0)

inputs = torch.stack([pad(x) for x, _ in pairs])
targets = torch.stack([pad(y) for _, y in pairs])

train_ds = torch.utils.data.TensorDataset(inputs, targets)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Veja um par

In [ ]:
print(pairs[1])

# Definição do modelo Seq2Seq com GRU

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        outputs, h = self.gru(x)
        return outputs, h   # <--- ESSENCIAL

In [ ]:
class LuongAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, decoder_hidden, encoder_outputs):
        """
        decoder_hidden: (B, 1, H)
        encoder_outputs: (B, S, H)

        Retorna:
          context: (B, 1, H)
          attn_weights: (B, 1, S)
        """

        # score = h_t · h_s^T
        # (B, 1, H) x (B, H, S) -> (B, 1, S)
        attn_scores = torch.bmm(decoder_hidden, encoder_outputs.transpose(1, 2))

        attn_weights = F.softmax(attn_scores, dim=-1)  # normaliza nos steps da source

        # context = soma ponderada
        # (B, 1, S) x (B, S, H) -> (B, 1, H)
        context = torch.bmm(attn_weights, encoder_outputs)

        return context, attn_weights

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.attn = LuongAttention()

        # Luong concat: concatena hidden + context
        self.fc = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, x, h, encoder_outputs):
        """
        x: tokens anteriores corretos  (B, T)
        h: estado inicial do decoder   (1, B, H)
        encoder_outputs: todos os h_s  (B, S, H)
        """
        x = self.embed(x)  # (B, T, E)

        outputs = []
        seq_len = x.size(1)
        hidden = h

        for t in range(seq_len):
            inp = x[:, t:t+1]  # (B, 1, E)

            out_t, hidden = self.gru(inp, hidden)   # out_t: (B,1,H)

            # Atenção
            context, attn_w = self.attn(out_t, encoder_outputs)

            # concatenação [out_t ; context]
            combined = torch.cat([out_t, context], dim=-1)

            logits = self.fc(combined)  # (B,1,V)
            outputs.append(logits)

        outputs = torch.cat(outputs, dim=1)  # (B, T, V)
        return outputs, hidden


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        encoder_outputs, h = self.encoder(src)
        logits, _ = self.decoder(tgt[:, :-1], h, encoder_outputs)
        return logits

# Código para usar o modelo treinado: inferência

In [ ]:
def decode_step(decoder, token, h, encoder_outputs):
    """
    Executa um passo de decodificação:
    - token: tensor (B,1)
    - h: estado oculto do decoder (1,B,H)
    - encoder_outputs: (B,S,H)
    """
    logits, h = decoder(token, h, encoder_outputs)  # (B,1,V)
    next_token = logits[:, -1, :].argmax(-1, keepdim=True)  # (B,1)
    return next_token, h


def predict(model, seq, max_len=10):
    model.eval()
    with torch.no_grad():
        # codifica entrada
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)

        # encoder agora retorna (encoder_outputs, h)
        encoder_outputs, h = model.encoder(src)

        # token inicial (ex: espaço ou <sos>)
        token = torch.tensor([[vocab[' ']]], dtype=torch.long, device=device)

        seq_invertida = []
        for _ in range(max_len):
            token, h = decode_step(model.decoder, token, h, encoder_outputs)
            seq_invertida.append(token.item())

        return decode(seq_invertida)


In [ ]:
def compara(seq_ori, seq_inv):
  res = 0;
  for i in range(len(seq_ori)):
    if(seq_ori[i] == seq_inv[(len(seq_ori)-1)-i]): res += 1
  return res/len(seq_ori)

# Preparação para treino

In [ ]:
emb_size = 32
hidden_size = 64
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' ']) # ignora o pad: " "
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

# Execução do treino

In [ ]:
for epoch in range(10):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)
        opt.zero_grad()
        logits = model(xb, yb)
        loss = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1))
        loss.backward()
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_dl):.4f}")

# Vamos testar

In [ ]:
acertos = []

for _ in range(100):
    s = random_seq()
    pred = predict(model, s, max_len=len(s))
    acc = compara(s, pred)
    acertos.append(acc)
    print(f"{s} -> {pred} --- Acertos = {acc*100:.2f}%")

# média geral
media_geral = sum(acertos) / len(acertos)
print(f"\n🎯 Média geral de acertos: {media_geral*100:.2f}%")


In [ ]:
def get_attention_and_prediction(model, seq):
    model.eval()
    with torch.no_grad():
        src = pad(encode(seq)).unsqueeze(0).to(device)
        encoder_outputs, h = model.encoder(src)

        token = torch.tensor([[vocab[' ']]], device=device)

        attn_list = []
        pred_tokens = []

        for _ in range(len(seq)):
            # EMBEDDING
            emb = model.decoder.embed(token)

            # GRU
            out_t, h = model.decoder.gru(emb, h)

            # ATENÇÃO
            context, attn_w = model.decoder.attn(out_t, encoder_outputs)
            attn_list.append(attn_w.squeeze(0))  # (1,S)

            # CONCAT
            combined = torch.cat([out_t, context], dim=-1)

            # FC
            logits = model.decoder.fc(combined)

            # PRÓXIMO TOKEN
            next_token = logits.argmax(-1)
            pred_tokens.append(int(next_token))

            token = next_token

        attn_matrix = torch.cat(attn_list, dim=0).cpu()  # (T_dec, T_enc)
        pred = decode(pred_tokens)

        return attn_matrix, pred




def plot_attention_matrix(attn_matrix, source_seq, predicted_seq):
    A = attn_matrix.numpy()

    fig, ax = plt.subplots(figsize=(8, 6))
    cax = ax.matshow(A, cmap="viridis")

    fig.colorbar(cax)

    # Labels dos eixos
    ax.set_xticks(range(len(source_seq)))
    ax.set_yticks(range(len(predicted_seq)))

    ax.set_xticklabels(list(source_seq))
    ax.set_yticklabels(list(predicted_seq))

    ax.set_xlabel("Sequência de Entrada (Source)")
    ax.set_ylabel("Sequência de Saída (Target)")

    # Insere os valores dentro das células
    for i in range(A.shape[0]):      # target
        for j in range(A.shape[1]):  # source
            ax.text(j, i, f"{A[i,j]:.2f}",
                    ha="center", va="center", color="white")

    plt.title("Mapa de Calor dos Pesos de Atenção")
    plt.show()


In [ ]:
seq = "abcd"

attn_matrix, pred = get_attention_and_prediction(model, seq)

print("Predição:", pred)

plot_attention_matrix(attn_matrix, seq, pred)


# Exercício
Compare o resultado do uso do encoder-decoder com atenção com o encoder-decoder sem atenção.

In [ ]:
"""Comparando os modelos com atenção e sem atenção com os seguintes parâmetros:
número de dados de treino = 50000,
emb_size = 32,
hidden_size = 64,
número de épocas = 20,
número de dados de testes = 100

Modelo com atenção:
bdacb -> bcadb --- Acertos = 100.00%
bbdad -> dadbd --- Acertos = 80.00%
ccaab -> baaca --- Acertos = 80.00%
bbdba -> abdbb --- Acertos = 100.00%
bbdbd -> dbdbd --- Acertos = 80.00%
cdcda -> adcdc --- Acertos = 100.00%
acaaa -> aaaac --- Acertos = 60.00%
acaab -> baaaa --- Acertos = 80.00%
ddaad -> dadad --- Acertos = 60.00%
adbab -> babda --- Acertos = 100.00%
dcaab -> baaad --- Acertos = 80.00%
bdbcb -> bcbdb --- Acertos = 100.00%
dddcd -> ddcdd --- Acertos = 60.00%
adcab -> badac --- Acertos = 40.00%
bcbdb -> bdbcb --- Acertos = 100.00%
caddb -> bddac --- Acertos = 100.00%
cddba -> abddc --- Acertos = 100.00%
bdaad -> dadab --- Acertos = 60.00%
abcbc -> cbcba --- Acertos = 100.00%
bcbad -> dabcb --- Acertos = 100.00%
cbcca -> accbc --- Acertos = 100.00%
ddbcd -> dcdbd --- Acertos = 60.00%
cadad -> daddc --- Acertos = 80.00%
cbacd -> dcacc --- Acertos = 80.00%
dbbda -> adbdd --- Acertos = 80.00%
dbcda -> adcdd --- Acertos = 80.00%
bacdc -> cdcbd --- Acertos = 60.00%
bdabc -> cbadb --- Acertos = 100.00%
bbddc -> cddbd --- Acertos = 80.00%
cbadc -> cdabc --- Acertos = 100.00%
abdaa -> aadba --- Acertos = 100.00%
dcbdd -> ddbdc --- Acertos = 60.00%
bbcbb -> bbcbb --- Acertos = 100.00%
bcaba -> ababc --- Acertos = 60.00%
bbcdb -> bdcbb --- Acertos = 100.00%
bdcab -> bacdb --- Acertos = 100.00%
dbbab -> babbd --- Acertos = 100.00%
adcaa -> aacda --- Acertos = 100.00%
cacdc -> cdcca --- Acertos = 60.00%
dbbcb -> bcbbd --- Acertos = 100.00%
cbccd -> dccdc --- Acertos = 80.00%
badba -> abdab --- Acertos = 100.00%
ddcac -> cadcd --- Acertos = 60.00%
abdac -> cadab --- Acertos = 60.00%
addcd -> ddcdd --- Acertos = 40.00%
daddc -> cdddd --- Acertos = 80.00%
ccbab -> babcc --- Acertos = 100.00%
aadba -> abdaa --- Acertos = 100.00%
adabd -> dbadd --- Acertos = 80.00%
bdbab -> babdb --- Acertos = 100.00%
aadbc -> cbdaa --- Acertos = 100.00%
bdbbc -> cbbdb --- Acertos = 100.00%
ddccc -> ccdcd --- Acertos = 60.00%
dbacd -> dcdab --- Acertos = 40.00%
dacba -> abcad --- Acertos = 100.00%
acabb -> babca --- Acertos = 60.00%

🎯 Média geral de acertos: 79.40%


Modelo sem atenção:
bcddb -> dbbcd --- Acertos = 20.00%
bccbc -> ccbdc --- Acertos = 20.00%
cccaa -> acacd --- Acertos = 40.00%
aadbc -> cdbcd --- Acertos = 20.00%
bbccc -> cccbd --- Acertos = 80.00%
cdacc -> ccdcd --- Acertos = 40.00%
baccd -> dcccd --- Acertos = 60.00%
ccacd -> dcccd --- Acertos = 60.00%
cadbc -> cdbcd --- Acertos = 20.00%
cdaac -> caddc --- Acertos = 80.00%
bbdba -> abddc --- Acertos = 60.00%
dbbad -> ddabc --- Acertos = 40.00%
dbccd -> dccdb --- Acertos = 60.00%
acbab -> babcd --- Acertos = 80.00%
bddbc -> cdbdc --- Acertos = 40.00%
bbbdd -> ddbdc --- Acertos = 60.00%
dbdbb -> bbddc --- Acertos = 60.00%
adcdc -> cddcc --- Acertos = 40.00%
cbdbd -> ddbcb --- Acertos = 20.00%
dbaca -> accdd --- Acertos = 60.00%
caaac -> cadac --- Acertos = 80.00%
cacbc -> ccbdc --- Acertos = 40.00%
cdbca -> cadbb --- Acertos = 0.00%
dbbdb -> bdbdc --- Acertos = 60.00%
cacab -> bacdc --- Acertos = 80.00%
bbccc -> cccbd --- Acertos = 80.00%
cbdac -> cdadc --- Acertos = 40.00%
bddcd -> dcddc --- Acertos = 80.00%
bacbc -> ccbdb --- Acertos = 40.00%
acaaa -> aacad --- Acertos = 40.00%
dcdbb -> bdbcb --- Acertos = 40.00%
babbd -> dbbdc --- Acertos = 60.00%
ddaac -> caddd --- Acertos = 80.00%
cabac -> caddc --- Acertos = 60.00%
dddbd -> ddbcd --- Acertos = 40.00%
aabaa -> aaabc --- Acertos = 40.00%
cbbad -> ddacb --- Acertos = 20.00%
ccabb -> bbcdb --- Acertos = 40.00%
cacbc -> ccbdc --- Acertos = 40.00%
aaaad -> dadca --- Acertos = 60.00%
bdcdc -> cdcdd --- Acertos = 80.00%
cadca -> cadda --- Acertos = 20.00%
bddab -> bdaad --- Acertos = 20.00%
abdad -> ddadc --- Acertos = 20.00%
ddacd -> dcdcd --- Acertos = 60.00%
adabd -> ddbba --- Acertos = 40.00%
bbbcb -> bcbcd --- Acertos = 60.00%
addaa -> adaac --- Acertos = 20.00%
dcbbb -> bbdcb --- Acertos = 60.00%
cccac -> ccdaa --- Acertos = 20.00%
dbbab -> babdb --- Acertos = 60.00%
dadbc -> cdbdc --- Acertos = 20.00%
cadcd -> dcdcd --- Acertos = 60.00%
dccbc -> ccbdc --- Acertos = 20.00%
cccca -> caccd --- Acertos = 40.00%
bdacc -> ccddc --- Acertos = 60.00%
cbacd -> dccdc --- Acertos = 60.00%
abcaa -> acaab --- Acertos = 20.00%
bbbad -> ddabb --- Acertos = 60.00%
abdba -> abddc --- Acertos = 60.00%
aabcc -> ccdcb --- Acertos = 40.00%
baddd -> ddddc --- Acertos = 60.00%
bdadc -> dcdca --- Acertos = 0.00%
adcca -> cadcc --- Acertos = 0.00%
bacbb -> bbcdc --- Acertos = 60.00%
bdcdc -> cdcdd --- Acertos = 80.00%
ddbdc -> cddbd --- Acertos = 60.00%
bbddb -> dbbdb --- Acertos = 20.00%
ddbad -> ddadc --- Acertos = 40.00%
acdaa -> aadcc --- Acertos = 80.00%
bddcd -> dcddc --- Acertos = 80.00%
cbdba -> abdcd --- Acertos = 60.00%
cbbbc -> cbdcb --- Acertos = 40.00%
ccdab -> bdacc --- Acertos = 60.00%
dddac -> cdadd --- Acertos = 60.00%
bcdab -> bdaca --- Acertos = 40.00%
dbdba -> abddc --- Acertos = 60.00%
babab -> babbd --- Acertos = 60.00%
cdcbb -> bbcdd --- Acertos = 80.00%
cbaab -> baadc --- Acertos = 80.00%
bdadb -> dbbad --- Acertos = 0.00%
babab -> babbd --- Acertos = 60.00%
ccdcb -> cbdcb --- Acertos = 40.00%
daccc -> cccdd --- Acertos = 80.00%
abdbc -> cdbbc --- Acertos = 40.00%
cbccb -> cbbcd --- Acertos = 0.00%
ccaab -> abbcc --- Acertos = 40.00%
bcbad -> ddcab --- Acertos = 40.00%
dddac -> cdadd --- Acertos = 60.00%
abddc -> dccdd --- Acertos = 0.00%
bdccb -> cbdcb --- Acertos = 20.00%
dabda -> daaba --- Acertos = 0.00%
ddddc -> dcdcd --- Acertos = 40.00%
dbcdd -> ddcdc --- Acertos = 60.00%
bcaba -> abcac --- Acertos = 40.00%
bdbdd -> dddbc --- Acertos = 40.00%
cdbab -> badbc --- Acertos = 60.00%
abaaa -> aaada --- Acertos = 80.00%
bcccb -> cbcdb --- Acertos = 40.00%
bcabd -> dbcdd --- Acertos = 40.00%

🎯 Média geral de acertos: 46.80%
"""





In [ ]:
"""Comparando os modelos com atenção e sem atenção com os seguintes parâmetros:
número de dados de treino = 50000,
emb_size = 32,
hidden_size = 64,
número de épocas = 20,
número de dados de testes = 10000

Modelo com atenção:
🎯 Média geral de acertos: 77.32%

Modelo sem atenção:
🎯 Média geral de acertos: 57.28%
"""

